In [1]:
import numpy as np
import pandas as pd
# from configRun import serverDataPath
# from configOutcomes import classesPerOutcome
# from utils import getExcldSemHalfsemFromWks

serverDataPath = "."
scenario = "pre_lock_change_feats_to_lock_mh" #"pre_lock_change_feats_to_lock_mh" # pre_feats_to_post_lock_mh
outcome_status = "msrsr_m_lbl"
scaling = True
tenFolds = True

if not tenFolds:
    folderpath = "{0}/SENSOR_COMBINATIONS_{1}_{2}/".format(serverDataPath, outcome_status, scenario)
else:
    folderpath = "{0}/SENSOR_COMBINATIONS_{1}_{2}_10Folds/".format(serverDataPath, outcome_status, scenario)

fnamelike = "results_sensor_combinations_nestgbc{0}"+"_scaling{0}.csv".format(scaling)
n_est_list = [60, 80, 100, 120, 140, 160, 180, 200, 220]



classesPerOutcome = {
    u'phq9_score_lbl': [0,1],
    u'mfis_m_lbl': [0,1],
    u'psqi_total_lbl': [0,1],
    u'pss_m_lbl': [0,1],
    u'msrsr_m_lbl': [0,1]
}


In [2]:
if len(classesPerOutcome[outcome_status]) > 2:
    METRIC_STR = "error"
else:
    METRIC_STR = "f11"
#     METRIC_STR = "accuracy"
#     if tenFolds:
#         METRIC_STR = "accuracy"
best_acc_dict = {}
n_est_for_best_acc_dict = {}
for n_est in n_est_list:
    fname = folderpath + fnamelike.format(n_est)
    print (fname)
    result = pd.read_csv(fname, index_col=0)
    result = result.set_index("sensors_combined")
    for scomb in result.index:
        acc = result.loc[scomb][METRIC_STR]
        print ("metric val {0}".format(acc))
        if scomb in best_acc_dict.keys():
            print (best_acc_dict[scomb])
            if (METRIC_STR == "accuracy" and acc > best_acc_dict[scomb]) or (METRIC_STR == "error" and acc < best_acc_dict[scomb]) or (METRIC_STR == "f11" and acc > best_acc_dict[scomb]):
                best_acc_dict[scomb] = acc
                n_est_for_best_acc_dict[scomb] = n_est
            elif acc == best_acc_dict[scomb]:  # we prefer n_ests closer to 100
                if abs(n_est_for_best_acc_dict[scomb] - 100) > abs(n_est - 100):
                    best_acc_dict[scomb] = acc
                    n_est_for_best_acc_dict[scomb] = n_est
        else:
            best_acc_dict[scomb] = acc
            n_est_for_best_acc_dict[scomb] = n_est
            # result = result.drop(["n_estimators_gbc"], axis=1)
            # print (result)

# print (best_acc_dict)
# print (n_est_for_best_acc_dict)
# print (len(best_acc_dict.keys()))
final_result = pd.DataFrame()
for scomb in best_acc_dict.keys():
    best_n_est = n_est_for_best_acc_dict[scomb]
    best_fname = folderpath + fnamelike.format(best_n_est)
    # print ("{0}:{1}".format(scomb, best_n_est))
    result = pd.read_csv(best_fname, index_col=0)
    result = result.set_index("sensors_combined")
    row = pd.DataFrame(result.loc[scomb])
    final_result = final_result.append(row.T)

# print (final_result)
final_result["sensors_combined"] = final_result.index
prf_cols = []
for c in classesPerOutcome[outcome_status]:
    prf_cols = prf_cols + ["precision{0}".format(c), "recall{0}".format(c), "f1{0}".format(c)]
if len(classesPerOutcome[outcome_status]) > 2:
    final_result = final_result.sort_values(by=[METRIC_STR], ascending=True)
    cols = ["missing_data_thres", "pre_status", "scaling", "adaboost", "base_estimator", "n_estimators", "learning_rate", "n_estimators_gbc", "learning_rate_gbc", "sensors_combined", "num_sensors", "best_models", "N", "accuracy", "error"] + prf_cols
else:
    final_result = final_result.sort_values(by=[METRIC_STR], ascending=False)
    cols = ["missing_data_thres", "pre_status", "scaling", "adaboost", "base_estimator", "n_estimators", "learning_rate", "n_estimators_gbc", "learning_rate_gbc", "sensors_combined", "num_sensors", "best_models", "N", "accuracy", "mcc"] + prf_cols
final_result.to_csv(folderpath + "BestSensorCombinations.csv", columns=cols)

./SENSOR_COMBINATIONS_msrsr_m_lbl_pre_lock_change_feats_to_lock_mh_10Folds/results_sensor_combinations_nestgbc60_scalingTrue.csv
metric val 0.770476190476
metric val 0.876666666667
metric val 0.79
metric val 0.823333333333
metric val 0.550476190476
metric val 0.663333333333
metric val 0.700476190476
metric val 0.926666666667
metric val 0.876666666667
metric val 0.856666666667
metric val 0.63
metric val 0.571428571429
metric val 0.68380952381
metric val 0.737142857143
metric val 0.78
metric val 0.726666666667
metric val 0.566666666667
metric val 0.74380952381
metric val 0.705
metric val 0.726666666667
metric val 0.632857142857
metric val 0.73380952381
metric val 0.84
metric val 0.550952380952
metric val 0.693333333333
metric val 0.657619047619
metric val 0.873333333333
metric val 0.806666666667
metric val 0.843333333333
metric val 0.546666666667
metric val 0.614285714286
metric val 0.54
metric val 0.705
metric val 0.8
metric val 0.643333333333
metric val 0.695476190476
metric val 0.6557

metric val 0.654285714286
0.702857142857143
metric val 0.756666666667
0.7566666666666666
metric val 0.624285714286
0.665952380952381
metric val 0.557142857143
0.5871428571428571
metric val 0.712857142857
0.7128571428571429
metric val 0.685238095238
0.6852380952380953
metric val 0.596666666667
0.63
metric val 0.685714285714
0.6857142857142857
metric val 0.605714285714
0.6414285714285715
metric val 0.595238095238
0.6378571428571429
metric val 0.602142857143
0.5888095238095239
./SENSOR_COMBINATIONS_msrsr_m_lbl_pre_lock_change_feats_to_lock_mh_10Folds/results_sensor_combinations_nestgbc220_scalingTrue.csv
metric val 0.813333333333
0.8133333333333332
metric val 0.876666666667
0.8766666666666667
metric val 0.79
0.7899999999999999
metric val 0.843333333333
0.8433333333333334
metric val 0.61380952381
0.6138095238095238
metric val 0.696666666667
0.6966666666666667
metric val 0.690476190476
0.7004761904761905
metric val 0.926666666667
0.9266666666666664
metric val 0.876666666667
0.87666666666666

/Users/prerna/Library/Python/2.7/lib/python/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
